In [1]:
from dataclasses import dataclass
import ipywidgets as widgets
import functools
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import sys
sys.path.append("..")
from config_through_cond_norm import Config

%load_ext autoreload
%autoreload 2

class Viewer:
    def update_profile(self, condition, aptaid, widget):
        filepath = self.config.shapem_svg_profile_path(condition, aptaid)
        try:
            with open(filepath, "rb") as file:
                image = file.read()
                widget.value = image
        except:
            widget.value = b''
    
    def update_structure(self, condition, aptaid, widget):
        filepath = self.config.varna_path(condition, aptaid)
        try:
            with open(filepath, "rb") as file:
                image = file.read()
                widget.value = image
        except:
            widget.value = b''
    
    def update_structure_box(self, condition, aptaid, widget):
        
        children = []
        try:
            for i in range(1, 2):
                filepath = self.config.varna_path(condition, aptaid, index=i)
                with open(filepath, "r") as file:
                    image = file.read()
                    iw = widgets.HTML(value=f"<div style='height:800px;width:800px;'>{image}</div>")
                    children.append(iw)
        except:
            pass
        widget.children = children
            
    #def update_structure_box(self, condition, aptaid, widget):
    #    
    #    children = []
    #    try:
    #        for i in range(1, 2):
    #            filepath = self.config.varna_path(condition, aptaid, index=i)
    #            with open(filepath, "rb") as file:
    #                image = file.read()
    #                iw = widgets.Image(
    #                    value=image,
    #                    format='svg+xml',
    #                    width=600,
    #                    height=500,
    #                    )
    #                iw.layout.object_fit = 'contains'
    #                children.append(iw)
    #    except:
    #        pass
    #    widget.children = children
    #        
    def update_view(self, handler, names = 'All', type = 'change', force = False):
        # DeltaShape
        if (handler and handler['name'] == "index") or force:
            c1 = self.compare_w.value[0]
            c2 = self.compare_w.value[1]
            ap = self.aptamer_w.value
            
            self.update_profile(c1, ap, self.sm_profile1)
            self.update_profile(c2, ap, self.sm_profile2)
            
            filepath = self.config.deltashape_svg_path(c1, c2, ap)

            try:
                with open(filepath, "rb") as file:
                    image = file.read()
                    self.deltashape.value = image
            except:
                self.deltashape.value = b''
    
            #self.update_structure(c1, ap, self.structure1)
            #self.update_structure(c2, ap, self.structure2)
            self.update_structure_box(c1, ap, self.structurebox1)
            self.update_structure_box(c2, ap, self.structurebox2)
         
        
    def __init__(self, config: Config ):
        self.config : Config = config
        self.init_view()
        self.init_events()
        self.update_view(None, force=True)

    def init_view(self):
        
        self.base_path = widgets.Text(
            value='/mnt/CiTCoM-Traduction/2_projects/2021-decrypted/data',
            description='Base Path',
            disabled=False
        )


        self.aptamer_w = widgets.Dropdown(
            options=self.config.aptamers()
,
            description='Aptamer',
        )
        #self.condition1_w = widgets.Dropdown(
        #options={v: k for k, v in config.conditions.items()},
        #description='Condition 1',
        #)
    #
        #self.condition2_w = widgets.Dropdown(
        #    options={v: k for k, v in config.conditions.items()},
        #    description='Condition 2',
        #    value="aptamer-sam_1"
        #)
        
            
        self.compare_w = widgets.Dropdown(
               options={v: k for k, v in config.compares.items()},
            description='Conditions',
            value=('SAMAP-nosam_Mg', 'SAMAP-sam_1_Mg')
        )
        
        self.sm_profile1 = widgets.Image(
        format='svg+xml',
        width=500,
        height=500,
        )
        
        self.sm_profile2 = widgets.Image(
        format='svg+xml',
        width=500,
        height=500,
        )
        self.deltashape = widgets.Image(
        format='svg+xml',
        width=1000,
        height=800,
        )
        
        self.structurebox1 = widgets.VBox([], width=500, height=600)
        self.structurebox2 = widgets.VBox([], width=500, height=600)
        
        self.structure1 = widgets.Image(
        format='svg+xml',
        width=800,
        height=600,
            
        )
        
        self.structure2 = widgets.Image(
        format='svg+xml',
        width=800,
        height=600,
        )
    
        self.prev_w= widgets.Button(
            description='Previous'
        )
        self.next_w = widgets.Button(
            description='Next'
        )
        
        self.profile_visible = widgets.Checkbox(
            value=True,
            description='Show profile',
            disabled=False,
            indent=False
        )
        self.deltashape_visible = widgets.Checkbox(
            value=True,
            description='Show deltashape',
            disabled=False,
            indent=False
        )
        self.structure_visible = widgets.Checkbox(
            value=True,
            description='Show structure',
            disabled=False,
            indent=False
        )
        
        #.layout.visibility = 'hidden'
        self.view = widgets.VBox(
        [
            widgets.HBox([ self.compare_w,self.aptamer_w,self.prev_w,self.next_w,self.profile_visible, self.deltashape_visible, self.structure_visible]),
            widgets.HBox([self.sm_profile1,self.sm_profile2]),
            self.deltashape,
            #widgets.HBox([self.structure1, self.structure2])
            widgets.HBox([self.structurebox1, self.structurebox2])
        ]
        )

    def toggle_visibility(self, cb, dst, h, n = 'All', t = 'change'):
        if cb.value:
            dst.layout.display = 'block'
        else:
            dst.layout.display= 'none'
        
    def next_aps(self, h, n = 'All', t = 'change'):
        cur = self.aptamer_w.index
        self.aptamer_w.value = self.aptamer_w.options[0 if cur >= len(self.aptamer_w.options) - 1 else cur + 1]

    def prev_aps(self, h, n = 'All', t = 'change'):
        cur = self.aptamer_w.index
        self.aptamer_w.value = self.aptamer_w.options[len(self.aptamer_w.options) - 1 if cur == 0 else cur - 1]
    
    
    def init_events(self):
        self.aptamer_w.observe(functools.partial(self.update_view))
        #self.condition1_w.observe(functools.partial(self.update_view))
        #self.condition2_w.observe(functools.partial(self.update_view))
        self.compare_w.observe(functools.partial(self.update_view))
        
        self.profile_visible.observe(functools.partial(self.toggle_visibility, self.profile_visible, self.sm_profile1))
        self.profile_visible.observe(functools.partial(self.toggle_visibility, self.profile_visible, self.sm_profile2))
        self.deltashape_visible.observe(functools.partial(self.toggle_visibility, self.deltashape_visible, self.deltashape))
        #self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structure1))
        #self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structure2))
        self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structurebox1))
        self.structure_visible.observe(functools.partial(self.toggle_visibility, self.structure_visible, self.structurebox2))
        self.next_w.on_click(functools.partial(self.next_aps))
        self.prev_w.on_click(functools.partial(self.prev_aps))
        
    


<ipython-input-1-570662e4cbe8>:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:




#file = open("images/WidgetArch.png", "rb")
#image = file.read()
#widgets.Image(
#    value=image,
#    format='png',
#    width=300,
#    height=400,
#)

config = Config()
#config.base_path = "../../data"


Viewer(config).view